In [14]:
# Carregar a biblioteca googledrive
library(googledrive)

# Autenticar no Google Drive (se necessário)
# drive_auth() # Descomente esta linha se você ainda não estiver autenticado

# ID do arquivo no Google Drive
file_id <- "https://drive.google.com/file/d/1Efop2rCqtf-M9QJHcVCLoPaYR9nxA-xo/view?usp=sharing" # Substitua pelo ID real do seu arquivo

# Baixar o arquivo do Google Drive
downloaded_file_path <- drive_download(
  as_id(file_id), # Usa o ID do arquivo para baixar
  path = "read.csv", # Nome do arquivo local (opcional, padrão é o nome original)
  overwrite = TRUE, # Sobrescreve o arquivo local se já existir
  verbose = FALSE # Suprime a mensagem de download
)

# Ler o arquivo usando o caminho completo
data <- read.csv(downloaded_file_path$local_path)

In [38]:
# Carregar bibliotecas necessárias
if (!require("matrixcalc")) install.packages("matrixcalc")
library(lavaan)
library(NCA)
library(tidyr)
library(dplyr)
library(matrixcalc)

# Função para definir a especificação do modelo SEM
define_model_specification <- function() {
  '
    Motivation =~ Mot1 + Mot2 + Mot3
    Performance =~ Des1 + Des2 + Des3
    Performance ~ Motivation
  '
}

# Função para verificar e ajustar a matriz de covariância
check_and_adjust_covariance <- function(data) {
  cov_matrix <- cov(data)
  if (is.singular.matrix(cov_matrix)) {
    # Aplicar regularização se a matriz for singular
    cov_matrix <- cov_matrix + diag(nrow(cov_matrix)) * 1e-5
  }
  cov_matrix
}

# Função para compilar o modelo SEM
compile_model <- function(model_specification, data) {
  observed_vars <- c("Mot1", "Mot2", "Mot3", "Des1", "Des2", "Des3")
  missing_vars <- observed_vars[!observed_vars %in% names(data)]
  if (length(missing_vars) > 0) {
    stop("Variáveis observadas ausentes no conjunto de dados: ",
         paste(missing_vars, collapse = ", "))
  }

  cov_matrix <- check_and_adjust_covariance(data)

  # Capturar aviso da função sem()
  compiled_model <- tryCatch({
    lavaan::sem(model_specification, sample.cov = cov_matrix, sample.nobs = nrow(data))
  }, warning = function(w) {
    message("Aviso durante a compilação do modelo: ", w$message)
    return(NULL) # Retornar NULL se houver aviso
  })

  compiled_model
}

# Função para realizar NCA e resumir os resultados
perform_nca <- function(compiled_model, nca_type, nca_variables) {
  if (!is.null(compiled_model)) { # Verificar se o modelo foi compilado com sucesso
    NCA::nca(compiled_model, nca_type, nca_variables) %>% summary() %>% print()
  } else {
    message("A análise NCA não pode ser realizada porque o modelo SEM não foi compilado com sucesso.")
  }
}

# Ler o arquivo CSV
data <- read.csv("read.csv")

# Separar a coluna única em várias colunas e converter para numérico
data <- data %>%
  separate(col = "Mot1.Mot2.Mot3.Des1.Des2.Des3",
           into = c("Mot1", "Mot2", "Mot3", "Des1", "Des2", "Des3"),
           sep = ";") %>%
  mutate(across(everything(), as.numeric))

# Executar a análise NCA-SEM
model_spec <- define_model_specification()
compiled_model <- compile_model(model_spec, data)
perform_nca(compiled_model, "necessity", c("Motivation", "Performance"))

Aviso durante a compilação do modelo: lavaan->lav_lavaan_step11_estoptim():  
   Model estimation FAILED! Returning starting values.

A análise NCA não pode ser realizada porque o modelo SEM não foi compilado com sucesso.

